# **Homework 2: Phoneme Classification**


Objectives:
* Solve a classification problem with deep neural networks (DNNs).
* Understand recursive neural networks (RNNs).

If you have any questions, please contact the TAs via TA hours, NTU COOL, or email to mlta-2023-spring@googlegroups.com

# Download Data
Download data from google drive, then unzip it.

You should have
- `libriphone/train_split.txt`: training metadata
- `libriphone/train_labels`: training labels
- `libriphone/test_split.txt`: testing metadata
- `libriphone/feat/train/*.pt`: training feature
- `libriphone/feat/test/*.pt`:  testing feature

after running the following block.

> **Notes: if the google drive link is dead, you can download the data directly from [Kaggle](https://www.kaggle.com/c/ml2023spring-hw2/data) and upload it to the workspace.**


In [2]:
!pip install --upgrade gdown

# Main link
# !gdown --id '1N1eVIDe9hKM5uiNRGmifBlwSDGiVXPJe' --output libriphone.zip
!gdown --id '1qzCRnywKh30mTbWUEjXuNT2isOCAPdO1' --output libriphone.zip

!unzip -q libriphone.zip
!ls libriphone

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


D:\Anaconda3\envs\cv\lib\site-packages\gdown\__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Failed to retrieve file url:

	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses.
	Check FAQ in https://github.com/wkentaro/gdown?tab=readme-ov-file#faq.

You may still be able to access the file from the browser:

	https://drive.google.com/uc?id='1qzCRnywKh30mTbWUEjXuNT2isOCAPdO1'

but Gdown can't. Please check connections and permissions.
unzip:  cannot find either libriphone.zip or libriphone.zip.zip.
'ls' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���


# Some Utility Functions
**Fixes random number generator seeds for reproducibility.**

In [1]:
import numpy as np
import torch
import random

def same_seeds(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

**Helper functions to pre-process the training data from raw MFCC features of each utterance.**

A phoneme may span several frames and is dependent to past and future frames. \
Hence we concatenate neighboring phonemes for training to achieve higher accuracy. The **concat_feat** function concatenates past and future k frames (total 2k+1 = n frames), and we predict the center frame.

Feel free to modify the data preprocess functions, but **do not drop any frame** (if you modify the functions, remember to check that the number of frames are the same as mentioned in the slides)

In [2]:
import os
import torch
from tqdm import tqdm

def load_feat(path):
    feat = torch.load(path)
    return feat

def shift(x, n):
    if n < 0:
        left = x[0].repeat(-n, 1)
        right = x[:n]
    elif n > 0:
        right = x[-1].repeat(n, 1)
        left = x[n:]
    else:
        return x

    return torch.cat((left, right), dim=0)

def concat_feat(x, concat_n):
    assert concat_n % 2 == 1 # n must be odd
    if concat_n < 2:
        return x
    seq_len, feature_dim = x.size(0), x.size(1)
    x = x.repeat(1, concat_n)
    x = x.view(seq_len, concat_n, feature_dim).permute(1, 0, 2) # concat_n, seq_len, feature_dim
    mid = (concat_n // 2)
    for r_idx in range(1, mid+1):
        x[mid + r_idx, :] = shift(x[mid + r_idx], r_idx)
        x[mid - r_idx, :] = shift(x[mid - r_idx], -r_idx)

    return x.permute(1, 0, 2).view(seq_len, concat_n * feature_dim)

def preprocess_data(split, feat_dir, phone_path, concat_nframes, train_ratio=0.8, random_seed=1213):
    class_num = 41 # NOTE: pre-computed, should not need change

    if split == 'train' or split == 'val':
        mode = 'train'
    elif split == 'test':
        mode = 'test'
    else:
        raise ValueError('Invalid \'split\' argument for dataset: PhoneDataset!')

    label_dict = {}
    if mode == 'train':
        for line in open(os.path.join(phone_path, f'{mode}_labels.txt')).readlines():
            line = line.strip('\n').split(' ')
            label_dict[line[0]] = [int(p) for p in line[1:]]

        # split training and validation data
        usage_list = open(os.path.join(phone_path, 'train_split.txt')).readlines()
        random.seed(random_seed)
        random.shuffle(usage_list)
        train_len = int(len(usage_list) * train_ratio)
        usage_list = usage_list[:train_len] if split == 'train' else usage_list[train_len:]

    elif mode == 'test':
        usage_list = open(os.path.join(phone_path, 'test_split.txt')).readlines()

    usage_list = [line.strip('\n') for line in usage_list]
    print('[Dataset] - # phone classes: ' + str(class_num) + ', number of utterances for ' + split + ': ' + str(len(usage_list)))

    max_len = 3000000
    X = torch.empty(max_len, 39 * concat_nframes)
    if mode == 'train':
        y = torch.empty(max_len, dtype=torch.long)

    idx = 0
    for i, fname in tqdm(enumerate(usage_list)):
        feat = load_feat(os.path.join(feat_dir, mode, f'{fname}.pt'))
        cur_len = len(feat)
        feat = concat_feat(feat, concat_nframes)
        if mode == 'train':
          label = torch.LongTensor(label_dict[fname])

        X[idx: idx + cur_len, :] = feat
        if mode == 'train':
          y[idx: idx + cur_len] = label

        idx += cur_len

    X = X[:idx, :]
    if mode == 'train':
      y = y[:idx]

    print(f'[INFO] {split} set')
    print(X.shape)
    if mode == 'train':
      print(y.shape)
      return X, y
    else:
      return X


# Dataset

In [4]:
import torch
from torch.utils.data import Dataset

class LibriDataset(Dataset):
    def __init__(self, X, y=None):
        self.data = X
        if y is not None:
            self.label = torch.LongTensor(y)
        else:
            self.label = None

    def __getitem__(self, idx):
        if self.label is not None:
            return self.data[idx], self.label[idx]
        else:
            return self.data[idx]

    def __len__(self):
        return len(self.data)


# Model
Feel free to modify the structure of the model.

In [5]:
import torch.nn as nn

class BasicBlock(nn.Module):
    def __init__(self, input_dim, output_dim, batchnorm=False, dropout=0.0):
        super(BasicBlock, self).__init__()

        # TODO: apply batch normalization and dropout for strong baseline.
        # Reference: https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm1d.html (batch normalization)
        #       https://pytorch.org/docs/stable/generated/torch.nn.Dropout.html (dropout)
        self.block = nn.Sequential(
            nn.Linear(input_dim, output_dim),
            nn.ReLU(),
        )
        if dropout:
            self.block.append(nn.Dropout(dropout))
        if batchnorm:
            self.block.append(nn.BatchNorm1d(output_dim))

    def forward(self, x):
        x = self.block(x)
        return x


class Classifier(nn.Module):
    def __init__(self, input_dim, output_dim=41, hidden_layers=1, hidden_dim=256):
        super(Classifier, self).__init__()

        self.fc = nn.Sequential(
            BasicBlock(input_dim, hidden_dim),
            *[BasicBlock(hidden_dim, hidden_dim) for _ in range(hidden_layers)],
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x):
        x = self.fc(x)
        return x
    
class ClassifierV2(nn.Module):
    def __init__(self, input_dim, output_dim=41, hidden_layers=1, hidden_dim=256, dropout=0.0):
        super().__init__()

        self.fc = nn.Sequential(
            BasicBlock(input_dim, hidden_dim),
            *[BasicBlock(hidden_dim, hidden_dim, batchnorm=True, dropout=dropout) for _ in range(hidden_layers)],
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x):
        x = self.fc(x)
        return x

In [6]:
import torch.nn as nn

class Classifier_GRU(nn.Module):
    """包含GRU模型的分类器"""
    def __init__(self, input_dim, 
                 output_dim=41, 
                 hidden_dim=1024, 
                 num_layers=2, 
                 bidirectional=False, 
                 dropout=0.0,
                 mlp_layers=[512, 256]):
        super().__init__()
        self.bidirectional = bidirectional
        self.gru = nn.GRU(input_size=input_dim, 
                          hidden_size=hidden_dim, 
                          num_layers=num_layers, 
                          batch_first=True,
                          dropout=dropout,
                          bidirectional=bidirectional)
        
        self.seq_len = None
        
        self.fc = []
        gru_output_dim = hidden_dim * 2 if bidirectional else hidden_dim
        for i in range(len(mlp_layers)):
            if i == 0:
                self.fc.append(nn.Linear(gru_output_dim, mlp_layers[i]))
            else:
                self.fc.append(nn.Linear(mlp_layers[i-1], mlp_layers[i]))
            self.fc.append(nn.ReLU())
        self.fc.append(nn.Linear(mlp_layers[-1], output_dim))
        self.fc = nn.Sequential(*self.fc)
        
    def forward(self, x):
        assert self.seq_len is not None
        batch_size, feature_size = x.shape
        assert feature_size % self.seq_len == 0
        x = x.view(batch_size, self.seq_len, feature_size // self.seq_len)
        output, _ = self.gru(x)
        x = self.fc(output[:, -1, :])  # [B, output_dim]
        return x

In [7]:
def count_parameters(model):
    """
    计算模型的总参数量

    参数:
    model (torch.nn.Module): PyTorch模型实例

    返回:
    int: 模型的总参数量
    """
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# Hyper-parameters

In [8]:
# data prarameters
# TODO: change the value of "concat_nframes" for medium baseline
concat_nframes = 5   # the number of frames to concat with, n must be odd (total 2k+1 = n frames)
train_ratio = 0.75   # the ratio of data used for training, the rest will be used for validation

# training parameters
seed = 1213          # random seed
batch_size = 2048        # batch size
num_epoch = 10         # the number of training epoch
learning_rate = 1e-4      # learning rate
model_path = './model.ckpt'  # the path where the checkpoint will be saved
# model parameters
# TODO: change the value of "hidden_layers" or "hidden_dim" for medium baseline
input_dim = 39 * concat_nframes  # the input dim of the model, you should not change the value
hidden_layers = 2          # the number of hidden layers
hidden_dim = 64           # the hidden dim

In [9]:
model_1 = Classifier(input_dim=input_dim, hidden_layers=8, hidden_dim=1024)
model_2 = Classifier(input_dim=input_dim, hidden_layers=2, hidden_dim=2048)
print(f'number of parameters of model_1: {count_parameters(model_1)}')
print(f'number of parameters of model_2: {count_parameters(model_2)}')

number of parameters of model_1: 8639529
number of parameters of model_2: 8878121


# Dataloader

In [10]:
from torch.utils.data import DataLoader
import gc

same_seeds(seed)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'DEVICE: {device}')

# preprocess data
train_X, train_y = preprocess_data(split='train', feat_dir='./libriphone/feat', phone_path='./libriphone', concat_nframes=concat_nframes, train_ratio=train_ratio, random_seed=seed)
val_X, val_y = preprocess_data(split='val', feat_dir='./libriphone/feat', phone_path='./libriphone', concat_nframes=concat_nframes, train_ratio=train_ratio, random_seed=seed)

# get dataset
train_set = LibriDataset(train_X, train_y)
val_set = LibriDataset(val_X, val_y)

# remove raw feature to save memory
del train_X, train_y, val_X, val_y
gc.collect()

# get dataloader
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)

DEVICE: cuda
[Dataset] - # phone classes: 41, number of utterances for train: 2571


2571it [00:01, 1334.37it/s]


[INFO] train set
torch.Size([1588590, 195])
torch.Size([1588590])
[Dataset] - # phone classes: 41, number of utterances for val: 858


858it [00:00, 1347.43it/s]

[INFO] val set
torch.Size([528204, 195])
torch.Size([528204])


In [11]:
for X, y in train_loader:
    print(X.shape)
    print(y.shape)
    break

torch.Size([2048, 195])
torch.Size([2048])


# Trainer

In [12]:
from torch.optim.lr_scheduler import PolynomialLR, StepLR

# train function
def train(num_epoch, model, optimizer, criterion, train_loader, val_loader, device, model_path, lr_scheduler=None):
    best_acc = 0.0
    for epoch in range(num_epoch):
        train_acc = 0.0
        train_loss = 0.0
        val_acc = 0.0
        val_loss = 0.0
        
        print(f'learning_rate: {lr_scheduler.get_last_lr()[0]}')
        # training
        model.train() # set the model to training mode
        for i, batch in enumerate(tqdm(train_loader)):
            features, labels = batch
            features = features.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(features)
            
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            _, train_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
            train_acc += (train_pred.detach() == labels.detach()).sum().item()
            train_loss += loss.item()

        # validation
        model.eval() # set the model to evaluation mode
        with torch.no_grad():
            for i, batch in enumerate(tqdm(val_loader)):
                features, labels = batch
                features = features.to(device)
                labels = labels.to(device)
                outputs = model(features)

                loss = criterion(outputs, labels)

                _, val_pred = torch.max(outputs, 1)
                val_acc += (val_pred.cpu() == labels.cpu()).sum().item() # get the index of the class with the highest probability
                val_loss += loss.item()

        print(f'[{epoch+1:03d}/{num_epoch:03d}] Train Acc: {train_acc/len(train_set):3.5f} Loss: {train_loss/len(train_loader):3.5f} | Val Acc: {val_acc/len(val_set):3.5f} loss: {val_loss/len(val_loader):3.5f}')

        # if the model improves, save a checkpoint at this epoch
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), model_path)
            print(f'saving model with acc {best_acc/len(val_set):.5f}')

        if lr_scheduler:
            lr_scheduler.step()

def train_from_init(num_epoch, model, train_loader, val_loader, device, learning_rate, model_path):
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()
    scheduler = StepLR(optimizer, step_size=10, gamma=0.6)
    train(num_epoch, model, optimizer, criterion, train_loader, val_loader, device, model_path, scheduler)

# create model, define a loss function, and optimizer
# model = Classifier(input_dim=input_dim, hidden_layers=hidden_layers, hidden_dim=hidden_dim).to(device)
# optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# train(num_epoch, model, train_loader, val_loader, device, learning_rate, model_path)

# Start to train

In [16]:
model_1 = model_1.to(device)
model_2 = model_2.to(device)
print(f'Start to train model_1 on {device}.')
train_from_init(num_epoch, model_1, train_loader, val_loader, device, learning_rate, 'model_1.ckpt')
# print('-----------------------------------')
# print(f'Start to train model_2 on {device}.')
# train(num_epoch, model_2, train_loader, val_loader, device, learning_rate, 'model_2.ckpt')

Start to train model_1 on cuda.
learning_rate: 0.0001


100%|██████████| 258/258 [00:05<00:00, 49.41it/s]


[001/010] Train Acc: 0.39434 Loss: 2.14627 | Val Acc: 0.48664 loss: 1.76620
saving model with acc 0.48664
learning_rate: 0.0001


100%|██████████| 258/258 [00:05<00:00, 48.20it/s]


[002/010] Train Acc: 0.51804 Loss: 1.63657 | Val Acc: 0.53103 loss: 1.59208
saving model with acc 0.53103
learning_rate: 0.0001


...

[005/010] Train Acc: 0.58646 Loss: 1.38017 | Val Acc: 0.57190 loss: 1.44078
saving model with acc 0.57190
learning_rate: 0.0001


100%|██████████| 258/258 [00:05<00:00, 45.87it/s]


[006/010] Train Acc: 0.59883 Loss: 1.33523 | Val Acc: 0.57627 loss: 1.43087
saving model with acc 0.57627
learning_rate: 0.0001


  8%|▊         | 64/776 [00:02<00:28, 24.69it/s]


KeyboardInterrupt: 

In [22]:
model_3 = ClassifierV2(input_dim=input_dim, hidden_layers=8, hidden_dim=1024, dropout=0.25).to(device)
model_4 = ClassifierV2(input_dim=input_dim, hidden_layers=8, hidden_dim=1024, dropout=0.5).to(device)
model_5 = ClassifierV2(input_dim=input_dim, hidden_layers=8, hidden_dim=1024, dropout=0.75).to(device)
print(f'Start to train model_3 on {device}.')
train(num_epoch, model_3, train_loader, val_loader, device, learning_rate, 'model_3.ckpt')
print('-----------------------------------')
print(f'Start to train model_4 on {device}.')
train(num_epoch, model_4, train_loader, val_loader, device, learning_rate, 'model_4.ckpt')
print('-----------------------------------')
print(f'Start to train model_5 on {device}.')
train(num_epoch, model_5, train_loader, val_loader, device, learning_rate, 'model_5.ckpt')

Start to train model_3 on cuda.


100%|██████████| 1032/1032 [00:05<00:00, 201.97it/s]


[001/010] Train Acc: 0.46160 Loss: 1.88658 | Val Acc: 0.52039 loss: 1.61677
saving model with acc 0.52039


100%|██████████| 1032/1032 [00:05<00:00, 189.55it/s]


[002/010] Train Acc: 0.51538 Loss: 1.63815 | Val Acc: 0.54076 loss: 1.53246
saving model with acc 0.54076


...

[008/010] Train Acc: 0.43018 Loss: 2.02155 | Val Acc: 0.45768 loss: 1.90302
saving model with acc 0.45768


100%|██████████| 1032/1032 [00:04<00:00, 208.97it/s]


[009/010] Train Acc: 0.43755 Loss: 1.99804 | Val Acc: 0.46303 loss: 1.88218
saving model with acc 0.46303


100%|██████████| 1032/1032 [00:04<00:00, 211.85it/s]

[010/010] Train Acc: 0.44367 Loss: 1.98042 | Val Acc: 0.47175 loss: 1.85445
saving model with acc 0.47175


In [23]:
model_3 = ClassifierV2(input_dim=input_dim, hidden_layers=8, hidden_dim=1024, dropout=0.25).to(device)
print(f'Start to train model_3 on {device}.')
train(50, model_3, train_loader, val_loader, device, learning_rate, 'model_3.ckpt')

Start to train model_3 on cuda.


100%|██████████| 1032/1032 [00:04<00:00, 206.94it/s]


[001/050] Train Acc: 0.46064 Loss: 1.89040 | Val Acc: 0.52081 loss: 1.62239
saving model with acc 0.52081


100%|██████████| 1032/1032 [00:04<00:00, 210.22it/s]


[002/050] Train Acc: 0.51576 Loss: 1.63735 | Val Acc: 0.53954 loss: 1.53431
saving model with acc 0.53954


...

[048/050] Train Acc: 0.64753 Loss: 1.13318 | Val Acc: 0.57639 loss: 1.42443


100%|██████████| 1032/1032 [00:05<00:00, 201.61it/s]


[049/050] Train Acc: 0.64875 Loss: 1.12926 | Val Acc: 0.57542 loss: 1.42798


100%|██████████| 1032/1032 [00:05<00:00, 194.35it/s]

[050/050] Train Acc: 0.64949 Loss: 1.12580 | Val Acc: 0.57555 loss: 1.43203


In [14]:
model_7 = Classifier_GRU(input_dim=input_dim // concat_nframes, 
                         hidden_dim=1024, 
                         bidirectional=True, 
                         num_layers=4, 
                         dropout=0.5,
                         mlp_layers=[1024, 512, 32]).to(device)
model_7.seq_len = concat_nframes
print(f'Start to train model_7 on {device}.')
# optimizer = torch.optim.Adam(model_7.parameters(), lr=learning_rate)
# criterion = nn.CrossEntropyLoss()
# scheduler = StepLR(optimizer, step_size=10, gamma=0.6)
# train(50, model_7, optimizer, criterion, train_loader, val_loader, device, 'model_7.ckpt', scheduler)
train_from_init(20, model_7, train_loader, val_loader, device, learning_rate, 'model_7.ckpt')

Start to train model_7 on cuda.
learning_rate: 0.0001


 51%|█████     | 131/258 [00:33<00:32,  3.93it/s]


KeyboardInterrupt: 

In [8]:
del train_set, val_set
del train_loader, val_loader
gc.collect()

0

# Testing
Create a testing dataset, and load model from the saved checkpoint.

In [ ]:
# load data
test_X = preprocess_data(split='test', feat_dir='./libriphone/feat', phone_path='./libriphone', concat_nframes=concat_nframes)
test_set = LibriDataset(test_X, None)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

[Dataset] - # phone classes: 41, number of utterances for test: 857


0it [00:00, ?it/s]<ipython-input-3-794249a209e1>:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  feat = torch.load(path)
857it [00:00, 1375.02it/s]

[INFO] test set
torch.Size([527364, 117])


In [ ]:
# load model
model = Classifier(input_dim=input_dim, hidden_layers=hidden_layers, hidden_dim=hidden_dim).to(device)
model.load_state_dict(torch.load(model_path))

<ipython-input-11-f7b7612de35f>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))


<All keys matched successfully>

Make prediction.

In [ ]:
pred = np.array([], dtype=np.int32)

model.eval()
with torch.no_grad():
    for i, batch in enumerate(tqdm(test_loader)):
        features = batch
        features = features.to(device)

        outputs = model(features)

        _, test_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        pred = np.concatenate((pred, test_pred.cpu().numpy()), axis=0)


100%|██████████| 1031/1031 [00:01<00:00, 518.37it/s]


Write prediction to a CSV file.

After finish running this block, download the file `prediction.csv` from the files section on the left-hand side and submit it to Kaggle.

In [ ]:
with open('prediction.csv', 'w') as f:
    f.write('Id,Class\n')
    for i, y in enumerate(pred):
        f.write('{},{}\n'.format(i, y))